In [ ]:
import os
os.environ["UNSLOTH_COMPILE_DISABLE"] = "1"
os.environ["UNSLOTH_DISABLE_FAST_GENERATION"] = "1"

In [ ]:
BASE_DIR = "/content/drive/MyDrive/smart_ecom_assistant"
!mkdir -p "$BASE_DIR"

In [ ]:
pip! install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
CONFIG = {
    "model_name": "meta-llama/Meta-Llama-3.1-8B-Instruct",
    "max_seq_length": 768,
    "epochs": 1,
    "batch_size": 1,
    "accum_steps": 16,
    "learning_rate": 2e-4,
    "warmup_ratio": 0.05,
    "weight_decay": 0.01,

    "train_path": "/content/drive/MyDrive/My_Machine_Learning/smart_E-commerce_assistant/samples/train_pairs_train.jsonl",
    "val_path":   "/content/drive/MyDrive/My_Machine_Learning/smart_E-commerce_assistant/samples/train_pairs_val.jsonl",
    "output_dir": "/content/drive/MyDrive/My_Machine_Learning/smart_E-commerce_assistant/models/lora-shopping-assistant",

    "seed": 42,
}
CONFIG



In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r                          = 16,
    target_modules             = ["q_proj", "v_proj"],
    lora_alpha                 = 32,
    lora_dropout               = 0.05,
    bias                       = "none",
    use_gradient_checkpointing = "unsloth",
    random_state               = CONFIG["seed"],
    use_rslora                 = False,
)

In [ ]:
from transformers import TrainingArguments

bf16_supported = is_bfloat16_supported()

training_args = TrainingArguments(
    output_dir                  = CONFIG["output_dir"],
    num_train_epochs            = CONFIG["epochs"],
    per_device_train_batch_size = CONFIG["batch_size"],
    gradient_accumulation_steps = CONFIG["accum_steps"],
    learning_rate               = CONFIG["learning_rate"],
    weight_decay                = CONFIG["weight_decay"],
    warmup_ratio                = CONFIG["warmup_ratio"],
    logging_steps               = 10,
    logging_first_step          = True,
    save_steps                  = 200,
    eval_strategy               = "epoch",
    save_total_limit            = 2,
    bf16                        = bf16_supported,
    fp16                        = not bf16_supported,
    optim                       = "paged_adamw_8bit",
    report_to                   = "none",
)

In [ ]:
def tokenize_batch(batch):
    enc = tokenizer(
        batch["text"],
        truncation   = True,
        max_length   = MAX_SEQ_LENGTH,
        padding      = "max_length",
    )
    labels = []
    pad_id = tokenizer.pad_token_id

    for seq in enc["input_ids"]:
        labels.append([
            (tok if tok != pad_id else -100)
            for tok in seq
        ])

    enc["labels"] = labels
    return enc

tokenized = dataset.map(
    tokenize_batch,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

train_dataset = tokenized["train"]
eval_dataset  = tokenized["validation"]

print(train_dataset[0].keys())

In [ ]:
bf16_supported = is_bfloat16_supported()

training_args = TrainingArguments(
    output_dir                  = CONFIG["output_dir"],
    num_train_epochs            = CONFIG["epochs"],
    per_device_train_batch_size = CONFIG["batch_size"],
    gradient_accumulation_steps = CONFIG["accum_steps"],
    learning_rate               = CONFIG["learning_rate"],
    logging_steps               = 5,
    logging_first_step          = True,
    save_steps                  = 200,
    eval_strategy               = "epoch",
    save_total_limit            = 2,
    bf16                        = bf16_supported,
    fp16                        = not bf16_supported,
    optim                       = "paged_adamw_8bit",
    lr_scheduler_type           = "cosine",
    warmup_ratio                = 0.05,
    report_to                   = "none",
)

In [ ]:
trainer = Trainer(
    model         = model,
    args          = training_args,
    train_dataset = train_dataset,
    eval_dataset  = eval_dataset,
)

trainer.train()

In [ ]:
save_dir = f'{CONFIG["output_dir"]}_final'
import os
os.makedirs(save_dir, exist_ok=True)

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print("Saved to:", save_dir)

In [ ]:
!zip -r /content/lora-shopping-assistant-final.zip "$save_dir"
from google.colab import files
files.download("/content/lora-shopping-assistant-final.zip")